In [ ]:
!pip install git+https://github.com/tweepy/tweepy.git
!pip install stop_words --use-pep517
!pip install vaderSentiment


import pandas as pd
from numpy import nan 
import tweepy
import datetime
import pytz
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.util import ngrams
from stop_words import get_stop_words
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk import bigrams

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tweepy/tweepy.git to /tmp/pip-req-build-tulzwlp_
  Running command git clone --filter=blob:none --quiet https://github.com/tweepy/tweepy.git /tmp/pip-req-build-tulzwlp_
  Resolved https://github.com/tweepy/tweepy.git to commit 0cd96b1918e5e920eb9f8fe4ba303ab5ec899c65
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 1.1 MB/s eta 0:00:00
  Created wheel for tweepy: filename=tweepy-4.12.1-py3-none-any.whl size=102442 sha256=af1e9d63fbf59742e87416a3c1338c74aa08f3feec1c672af21699e39dc3b5a8
  Stored in directory: /tmp/pip-ephem-wheel-cache-vdk9coe0/wheels/ad/05/51/a78f66d15b87f9c623d2f3afc4401660ac4219e526c787fb8b
Successfully built tweepy
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uni

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
company_name = input("Enter company name in which you are interested to invest: ")

Enter company name in which you are interested to invest: Coca-Cola Europacific Partners PLC


In [ ]:
print(f"Searching reviews for {company_name}, please wait...")
print("Source: https://finviz.com/")

df_finz = pd.read_csv("/content/drive/MyDrive/cyberCypher2.0_team42/finviz_sentiment.csv")

finz_positive = df_finz.apply(lambda x: True if x['company_name'] == company_name and x['sentiment'] == 'positive' else False, axis = 1)
finz_positive = len(finz_positive[finz_positive == True].index)

finz_negative = df_finz.apply(lambda x: True if x['company_name'] == company_name and x['sentiment'] == 'negative' else False, axis = 1)
finz_negative = len(finz_negative[finz_negative == True].index)

print(f'\nFound {finz_positive} positive articles and {finz_negative} negative articles')

finz_score = df_finz.loc[df_finz['company_name'] == company_name, 'compound'].mean()

Searching reviews for Coca-Cola Europacific Partners PLC, please wait...
Source: https://finviz.com/

Found 95 positive articles and 5 negative articles


In [ ]:
print(f"Searching reviews for {company_name}, please wait...")
print("Source: https://www.livemint.com/")

df_mint = pd.read_csv("/content/drive/MyDrive/cyberCypher2.0_team42/mint_sentiment.csv")

mint_positive = df_mint.apply(lambda x: True if x['Company_Name'] == company_name and x['sentiment'] == 'positive' else False, axis = 1)
mint_positive = len(mint_positive[mint_positive == True].index)

mint_negative = df_mint.apply(lambda x: True if x['Company_Name'] == company_name and x['sentiment'] == 'negative' else False, axis = 1)
mint_negative = len(mint_negative[mint_negative == True].index)

print(f'\nFound {mint_positive} positive articles and {mint_negative} negative articles')

mint_score = df_mint.loc[df_mint['Company_Name'] == company_name, 'compound'].mean()

Searching reviews for Coca-Cola Europacific Partners PLC, please wait...
Source: https://www.livemint.com/

Found 0 positive articles and 0 negative articles


In [ ]:
print(f"Searching reviews for {company_name}, please wait...")
print("Source: https://www.twitter.com/")

# Set up the API with your Twitter developer account's credentials
consumer_key = "xxxx"
consumer_secret = "xxxx"
access_token = "xxxx"
access_token_secret = "xxxx"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

stock_ticker = str(input('Enter the stock code: '))
stock_ticker_full = company_name
stock_hashtag = '#' + stock_ticker
data = []
usernames = ["wsj", "Reuters", "business","CNBC","WSJmarkets","livemint",
             "moneycontrolcom","NDTVProfit","bsindia","FinancialTimes",
             "ETMarkets","bqprime","Investopedia","BloombergTV","FinancialXpress",
             "NSEIndia","TOIBusiness","IIFL_Live","elearnmarkets","mystockedge"]

for username in usernames:
    tweets = api.user_timeline(screen_name=username, count=100)
    for tweet in tweets:
        if stock_ticker in tweet.text or stock_ticker_full in tweet.text:
            data.append(tweet.text)
            
tweets = api.search_tweets(q=stock_hashtag, count=100)

for tweet in tweets:
    data.append(tweet.text)

stop_words = list(stopwords.words('english')) + list(get_stop_words('en'))
pattern = '[0-9]'
cleaned_data = []

def generate_clean_ngrams(tweet_text, n):

    tweet_text = tweet_text.lower()
    tweet_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', tweet_text)
    tweet_text = re.sub(pattern, 'http', tweet_text)
    
    tokens = [token for token in tweet_text.split(' ') 
              if token != '' 
              and token not in stop_words 
              and 'http' not in token]
    
    output = list(ngrams(tokens, n))
    
    return output

for d in data:
    cleaned_data.append(generate_clean_ngrams(d, 2))

analyzer = SentimentIntensityAnalyzer()
score = []

for data in cleaned_data:
    for d in data:
        bigram_text = " ".join(d)
        scores = analyzer.polarity_scores(bigram_text)
        score.append(scores['compound'])
pe,ne = 0,0
for s in score:
  if s > 0:
    pe+=1
  elif s< 0:
    ne+=1

print(f'\nFound {pe} positive articles and {ne} negative articles')

try:
    average_score = sum(score) / len(score)
except ZeroDivisionError:
    average_score = 0

try:
    twitter_mean = sum(score) / len(score)
except ZeroDivisionError:
    twitter_mean = 0


Searching reviews for Coca-Cola Europacific Partners PLC, please wait...
Source: https://www.twitter.com/
Enter the stock code: CCEP

Found 20 positive articles and 1 negative articles


In [ ]:
import numpy as np

if np.isnan(mint_score):
  mint_score = 0

if np.isnan(twitter_mean):
  twitter_mean = 0

if np.isnan(finz_score):
  finz_score = 0

try:
  mean = (twitter_mean + mint_score + finz_score) / 3
except:
  mean = 0

if mean >=0:
  print(f'We recommend you to invest in {company_name}')
else:
  print(f'We DO NOT recommend you to invest in {company_name}')

We recommend you to invest in Coca-Cola Europacific Partners PLC
